In [1]:
import spleeter
import ffmpeg

import os
import subprocess

path = os.getcwd()

path_dir = 'C:\\Users\\dltkd\\!Find_Melody\\backup'
file_list = os.listdir(path_dir)

filename_list = []
filename_full_list = []

for i in range(0, len(file_list)):
    filename = file_list[i]
    if filename[-4:] == ".mp3":
        filename_list.append(filename)

In [2]:
len(filename_list)

1740

In [3]:
import random

genre = ['Ballade', 'HipHop', 'Rock', 'R&B', 'Classic', 'Soul', 'Idol']
country = ['Western', 'KPOP', 'CPOP', 'JPOP']
singer = ['Solo', 'Duo', 'Band', 'Group']

genre_list = []
country_list = []
singer_list = []

for i in range(0, len(filename_list)):
    genre_list.append(random.sample(genre, 1))
    country_list.append(random.sample(country, 1))
    singer_list.append(random.sample(singer, 1))

In [4]:
import pandas as pd

data = {'title': filename_list,
        'genre': genre_list,
        'country': country_list,
        'singer': singer_list,
        }

df = pd.DataFrame(data)
df

,title,genre,country,singer
0,(나알노) Bebe Rexha - Sabotage [가사해석].3gpp.mp3,[HipHop],[KPOP],[Solo]
1,(물음표 question mark) (Feat 최자 Choiza of 다이나믹듀오 ...,[Rock],[KPOP],[Band]
2,(심규선)아라리Arari【COVER by Guriri】.3gpp.mp3,[Soul],[KPOP],[Group]
3,(여자)아이들((G)I-DLE) - MY BAG (Choreography Pract...,[Ballade],[Western],[Solo]
4,(여자)아이들((G)I-DLE) - Nxde Official Music Video....,[Classic],[KPOP],[Solo]
...,...,...,...,...
1735,🔥빌보드 1위 X 멜론 1위의 조합 24kGoldn x sokodomo(소코도모)...,[R&B],[JPOP],[Group]
1736,🔥역대급 역주행🔥 𝗚𝗹𝗮𝘀𝘀 𝗔𝗻𝗶𝗺𝗮𝗹𝘀 - 𝗛𝗲𝗮𝘁 𝗪𝗮𝘃𝗲𝘀 [가사해석lyri...,[Rock],[JPOP],[Duo]
1737,🔥연패를 연승으로 바꿔줄 노래 Alan Walker(알렌 워커) x Benjami...,[R&B],[CPOP],[Duo]
1738,🚨비트부터 미쳤다 X Ambassadors - HEY CHILD (가사해석번역lyr...,[Classic],[Western],[Band]


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
counter_vector = CountVectorizer(ngram_range=(1,1))
c_vector_genre = counter_vector.fit_transform(df['genre'].apply(str))
c_vector_country = counter_vector.fit_transform(df['country'].apply(str))
c_vector_singer = counter_vector.fit_transform(df['singer'].apply(str))

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_genre = cosine_similarity(c_vector_genre, c_vector_genre).argsort()[:, ::-1]
print(similarity_genre)

[[   0 1242  410 ... 1115 1116  869]
 [1273 1306  404 ... 1125 1126    0]
 [ 934  379  632 ... 1131 1132    0]
 ...
 [1739  583  572 ... 1165 1166    0]
 [1739  429  454 ... 1134 1135    0]
 [1739  429  454 ... 1134 1135    0]]


In [7]:
similarity_genre.shape

(1740, 1740)

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_country = cosine_similarity(c_vector_country, c_vector_country).argsort()[:, ::-1]
print(similarity_country)

[[   0 1292 1350 ... 1072 1074  869]
 [   0 1292 1350 ... 1072 1074  869]
 [   0 1292 1350 ... 1072 1074  869]
 ...
 [1739 1251 1195 ... 1093 1094    0]
 [ 869 1256 1212 ... 1083 1084    0]
 [1739 1251 1195 ... 1093 1094    0]]


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_singer = cosine_similarity(c_vector_singer, c_vector_singer).argsort()[:, ::-1]
print(similarity_singer)

[[   0  929  520 ... 1075 1076  869]
 [1088  906 1404 ... 1079 1080    0]
 [1739 1229 1255 ... 1087 1088    0]
 ...
 [ 869  577 1342 ... 1088 1089    0]
 [1088  906 1404 ... 1079 1080    0]
 [1739 1229 1255 ... 1087 1088    0]]


In [27]:
def recommend_song_list_by_genre(df, song_title, top=30):
    target_song_index = df[df['title'].str.contains(song_title, na = False)].index.values[0]
    
    sim_index = similarity_genre[target_song_index, :top].reshape(-1)
    sim_index = sim_index[sim_index != target_song_index]
    
    result = df.iloc[sim_index][:10]
    return result

def recommend_song_list_by_country(df, song_title, top=30):
    target_song_index = df[df['title'].str.contains(song_title, na = False)].index.values[0]
    
    sim_index = similarity_country[target_song_index, :top].reshape(-1)
    sim_index = sim_index[sim_index != target_song_index]
    
    result = df.iloc[sim_index][:10]
    return result

def recommend_song_list_by_singer(df, song_title, top=30):
    target_song_index = df[df['title'].str.contains(song_title, na = False)].index.values[1]
    
    sim_index = similarity_singer[target_song_index, :top].reshape(-1)
    sim_index = sim_index[sim_index != target_song_index]
    
    result = df.iloc[sim_index][:10]
    return result

In [13]:
df.loc[1050]

title      [cover] vivi  PARED.3gpp.mp3
genre                         [Classic]
country                          [JPOP]
singer                            [Duo]
Name: 1050, dtype: object

In [14]:
recommend_song_list_by_genre(df, song_title = input("노래 제목을 입력하세요 "))

노래 제목을 입력하세요 vivi


,title,genre,country,singer
1739,🥀위험할 정도로 치명적인 노래 Nico Collins - Hate Me [가사 해석...,[Classic],[CPOP],[Group]
429,Howls Moving Castle - Merry go round of Life c...,[Classic],[JPOP],[Band]
454,I Need You (어떻게 지내).3gpp.mp3,[Classic],[JPOP],[Solo]
449,I Like Me Better.3gpp.mp3,[Classic],[KPOP],[Solo]
446,I Gotta Feeling.3gpp.mp3,[Classic],[JPOP],[Band]
1235,米津玄師 - LOSER Kenshi Yonezu.3gpp.mp3,[Classic],[KPOP],[Band]
443,I Don’t Wanna Live Forever (Fifty Shades Darke...,[Classic],[Western],[Solo]
441,I Dont Care.3gpp.mp3,[Classic],[Western],[Band]
1244,香水 瑛人 (Official Music Video).3gpp.mp3,[Classic],[JPOP],[Solo]
1250,가사 존나 슬프네🥹 Bruno Mars - When I Was Your Man [...,[Classic],[CPOP],[Duo]


In [15]:
recommend_song_list_by_country(df, song_title = input("노래 제목을 입력하세요 "))

노래 제목을 입력하세요 vivi


,title,genre,country,singer
480,Isac Elliot - Ghost (Lyric Video).3gpp.mp3,[Ballade],[JPOP],[Solo]
1444,봄여름가을겨울 (Still Life).3gpp.mp3,[HipHop],[JPOP],[Group]
607,Memories.3gpp.mp3,[Ballade],[JPOP],[Duo]
1431,버들개지 떨어지고 두견새 슬피 우네 (杨花落尽子规啼) - 황슬푸(黄诗扶) G2er....,[Ballade],[JPOP],[Group]
1433,버퍼링 Glitch Mode.3gpp.mp3,[HipHop],[JPOP],[Group]
1434,벌거벗은 마음 받아줘🍋 아이묭 - 벌거벗은 마음(裸の心 하다카노 코코로) [가사...,[Classic],[JPOP],[Solo]
609,METEOR.3gpp.mp3,[Classic],[JPOP],[Band]
243,Cold Blooded.3gpp.mp3,[Soul],[JPOP],[Band]
1438,변비.3gpp.mp3,[Rock],[JPOP],[Band]
611,MIC Drop.3gpp.mp3,[Rock],[JPOP],[Group]


In [16]:
recommend_song_list_by_singer(df, song_title = input("노래 제목을 입력하세요 "))

노래 제목을 입력하세요 vivi


,title,genre,country,singer
869,STEP.3gpp.mp3,[Idol],[Western],[Duo]
577,Love Yourself.3gpp.mp3,[Soul],[JPOP],[Duo]
1342,네가 떠나던 날 (当你走了 당니주료) - 우준 (吾尊 오존) 가사해석 번역 병음 ...,[R&B],[Western],[Duo]
1344,네자루의 MIC (Feat Room9 飛刀승우 Illinit).3gpp.mp3,[Classic],[CPOP],[Duo]
1345,노력에 보답받지 못해도❄️ Mrs Green Apple🍏 - 나라는 것(僕のこと) ...,[Ballade],[CPOP],[Duo]
1346,누구를 위한 삶인가 Whos Life Is This For (사생결단 ost).3...,[R&B],[Western],[Duo]
294,Dont Flex on me (Feat jayci yucca 김효은 Keem Hyo...,[Idol],[Western],[Duo]
1349,눈누난나 (NUNU NANA).3gpp.mp3,[Ballade],[KPOP],[Duo]
968,VILLAIN DIES.3gpp.mp3,[R&B],[KPOP],[Duo]
290,Doja Cat - Say So (Official Video).3gpp.mp3,[Idol],[CPOP],[Duo]


In [20]:
df.loc[df[df['title'].str.contains('way back home', na = False)].index.values[0]]

title      너를생각해 × way back home.3gpp.mp3
genre                              [Idol]
country                         [Western]
singer                             [Band]
Name: 1329, dtype: object

In [17]:
recommend_song_list_by_genre(df, song_title = input("노래 제목을 입력하세요 "))

노래 제목을 입력하세요 way back home


,title,genre,country,singer
869,STEP.3gpp.mp3,[Idol],[Western],[Duo]
364,GD & TOP - High High.3gpp.mp3,[Idol],[JPOP],[Group]
1274,광화문에서 At Gwanghwamun.3gpp.mp3,[Idol],[Western],[Duo]
1276,구두.3gpp.mp3,[Idol],[JPOP],[Band]
1279,구원찬 Ku One Chan - 01 너는 어떻게 (feat 백예린) (Offic...,[Idol],[KPOP],[Band]
393,Good Love (Feat BSK aka 김범수).3gpp.mp3,[Idol],[JPOP],[Band]
1292,그래서 그대는 (그래서 그대는).3gpp.mp3,[Idol],[KPOP],[Solo]
390,Good Day (Feat 팔로알토 Paloalto) (Prod 코드 쿤스트 COD...,[Idol],[JPOP],[Solo]
389,GOOD BOY.3gpp.mp3,[Idol],[KPOP],[Solo]
1304,꽃 길 (Flower Road).3gpp.mp3,[Idol],[KPOP],[Group]


In [21]:
df.loc[df[df['title'].str.contains('돌아오지마', na = False)].index.values[0]]

title      Heize (헤이즈) - 돌아오지마 (Feat 용준형 Of BEAST 비스트).3g...
genre                                                 [Soul]
country                                            [Western]
singer                                                 [Duo]
Name: 411, dtype: object

In [22]:
recommend_song_list_by_country(df, song_title = input("노래 제목을 입력하세요 "))

노래 제목을 입력하세요 돌아오지마


,title,genre,country,singer
869,STEP.3gpp.mp3,[Idol],[Western],[Duo]
1256,강산도 앞길을 막지 못하리라 (山止川行 shān zhĭ chuān xíng) - 부...,[R&B],[Western],[Group]
1212,冯提莫翻唱周深《大鱼》.3gpp.mp3,[Idol],[Western],[Band]
399,Habits (Stay High).3gpp.mp3,[R&B],[Western],[Duo]
400,Happier.3gpp.mp3,[HipHop],[Western],[Band]
410,Heavens Cloud.3gpp.mp3,[HipHop],[Western],[Solo]
412,Hello Future.3gpp.mp3,[R&B],[Western],[Group]
413,HeMeets - 01 Cecil Hotel 세실호텔 (Official Audio)...,[R&B],[Western],[Solo]
1194,ずっと真夜中でいいのに。『MILABO』MV（ZUTOMAYO - MILABO）.3gpp...,[Rock],[Western],[Solo]
414,HeMeets - 05 곡부득이소정가(Official Audio) with lyri...,[Rock],[Western],[Group]


In [26]:
df.loc[df[df['title'].str.contains('사랑했나봐', na = False)].index.values[0]]

title      It Must Have Been Love(사랑했나봐) - Jang Hye Jin [...
genre                                                 [Soul]
country                                               [KPOP]
singer                                                [Band]
Name: 484, dtype: object

In [28]:
recommend_song_list_by_singer(df, song_title = input("노래 제목을 입력하세요 "))

노래 제목을 입력하세요 사랑했나봐


,title,genre,country,singer
1088,[MV] Mad Clown(매드클라운) _ Fire(화) (Feat Jinsil(진...,[Idol],[CPOP],[Band]
906,thank u next.3gpp.mp3,[Rock],[Western],[Band]
1404,마크툽 (MAKTUB) - 오늘도 빛나는 너에게(To You My Light)(Fe...,[Soul],[KPOP],[Band]
914,The One That Got Away.3gpp.mp3,[HipHop],[Western],[Band]
913,The Middle.3gpp.mp3,[Rock],[JPOP],[Band]
1184,【そんなふいんきで歌ってみた】だれかの心臓になれたなら【ぐるたみん】.3gpp.mp3,[Ballade],[KPOP],[Band]
243,Cold Blooded.3gpp.mp3,[Soul],[JPOP],[Band]
910,The Chainsmokers & Coldplay - Something Just L...,[Idol],[JPOP],[Band]
517,KICK BACK.3gpp.mp3,[R&B],[CPOP],[Band]
239,Clean Bandit - Rockabye (한국어 가사해석자막).3gpp.mp3,[Soul],[CPOP],[Band]


In [30]:
df[df['title'].str.contains('vivi', na = False)].index.values[0]

1050